## Лабораторная № 3. Тематическое моделирование

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Выполните тематическое моделирование отзывов о банковских услугах помощью LDA (используйте библиотеку gensim) для одного из топ-5 банков по количеству отзывов. Для уменьшение времени вычисления можно взять выборку отзывов.


In [ ]:
!pip install pyLDAvis
!pip install pymorphy2

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def preprcesss(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = gensim.utils.simple_preprocess(text, deacc=True)

  return text

In [ ]:
data = pd.read_csv('/content/financial_review.csv')
# оставляю только тиньк
bank_name = data['bank_name'].value_counts().head(5).to_dict()
data_bank = data.loc[data['bank_name'] == list(bank_name.items())[2][0]]

data_bank_lemma = data_bank.copy()

data = data_bank['review'].values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
data_words = [preprcesss(text) for text in data]

bigram = gensim.models.Phrases(data_words, min_count=3, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
morph = pymorphy2.MorphAnalyzer()

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Py

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import spacy
! python -m spacy download ru_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load('ru_core_news_sm')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def lemma(text):
  texts_out = []
  for sent in text:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.is_alpha])
  return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_lemmatized = lemma(data_words_bigrams)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# словарь слово-индекс
id2word = corpora.Dictionary(data_lemmatized)
corpus = [id2word.doc2bow(review) for review in data_lemmatized]

print(id2word, '\n', corpus[101])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dictionary<16101 unique tokens: ['ps', 'адрес', 'банк', 'банкеоплатила', 'беременный']...> 
 [(0, 3), (2, 26), (15, 12), (17, 3), (18, 6), (22, 3), (27, 3), (63, 3), (100, 1), (108, 3), (109, 6), (159, 6), (165, 9), (180, 6), (195, 3), (222, 3), (249, 3), (293, 3), (325, 6), (376, 3), (389, 3), (398, 3), (448, 6), (579, 3), (611, 12), (642, 3), (710, 6), (757, 3), (781, 3), (878, 3), (921, 3), (945, 3), (961, 3), (998, 3), (1219, 3), (1341, 3), (1395, 3), (1484, 3), (1568, 3), (1829, 3), (1870, 3), (1906, 3), (2205, 3), (2874, 6), (2906, 3), (3454, 6), (3455, 3), (3456, 3), (3457, 3), (3458, 3), (3459, 3), (3460, 9), (3461, 3), (3462, 3), (3463, 3), (3464, 3), (3465, 3)]


In [ ]:
id2word[2] # просто интересно, что за слово встречается 26 раз :)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'банк'

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=15,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pprint(lda_model.print_topics())

[(0,
  '0.021*"захотеть" + 0.019*"ооо" + 0.017*"разочаровать" + 0.016*"самым" + '
  '0.015*"названивать" + 0.015*"меньше" + 0.014*"постоянный" + '
  '0.012*"навязывать" + 0.012*"скрыть" + 0.011*"пристав"'),
 (1,
  '0.000*"объективность" + 0.000*"международныи" + 0.000*"побег" + '
  '0.000*"перечеркнулось" + 0.000*"переводчик" + 0.000*"огорчить" + '
  '0.000*"получательница" + 0.000*"мучаетсяпалка" + 0.000*"кириллицеи" + '
  '0.000*"комсы"'),
 (2,
  '0.032*"валютный" + 0.011*"приятныи" + 0.003*"пустой" + 0.003*"дно" + '
  '0.002*"следствие" + 0.001*"бесплатнои" + 0.000*"терпеливо" + '
  '0.000*"собственный" + 0.000*"десяток" + 0.000*"подбор"'),
 (3,
  '0.027*"закон" + 0.015*"ответственность" + 0.015*"бки" + 0.015*"продать" + '
  '0.013*"право" + 0.013*"однажды" + 0.013*"семья" + 0.012*"автоматически" + '
  '0.012*"здоровье" + 0.012*"оно"'),
 (4,
  '0.020*"предприятие" + 0.009*"беседа" + 0.003*"тц" + 0.003*"значительный" + '
  '0.002*"оплате" + 0.001*"переплатить" + 0.000*"вымогательство

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('\nPerplexity: ', round(lda_model.log_perplexity(corpus), 3))

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', round(coherence_lda, 3))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -11.365

Coherence Score:  0.51


### Найдите оптимальное количество тем в соответствии с метриками: 1) Perplexity score 2) Coherence score. Есть ли различия?


In [ ]:
global coherence_list, model_list
coherence_list = []
model_list = []

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def find_model(start, finish):
  perplexity_min = 0
  coherence_min = 0.5
  theme_amount_p = 0
  theme_amount_c = 0

  for elem in range(start, finish):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=elem,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized,
                                        dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    if lda_model.log_perplexity(corpus) < perplexity_min:
      perplexity_min = lda_model.log_perplexity(corpus)
      theme_amount_p = elem
    if coherence_lda < coherence_min:
      coherence_min = coherence_lda
      theme_amount_c = elem

    coherence_list.append(coherence_lda)
    model_list.append(lda_model)

    print(f'{elem}: Perplexity = {round(lda_model.log_perplexity(corpus), 3)}. Coherence Score = {round(coherence_lda, 3)}')

  print(f'\n\nPerplexity of {theme_amount_p} themes: {round(perplexity_min, 3)}' + '\n' +
        f'Coherence Score of {theme_amount_c} themes: {round(coherence_min, 3)}\n')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
%time find_model(1, 15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1: Perplexity = -8.075. Coherence Score = 0.253
2: Perplexity = -8.086. Coherence Score = 0.301
3: Perplexity = -8.1. Coherence Score = 0.318
4: Perplexity = -8.138. Coherence Score = 0.387
5: Perplexity = -8.173. Coherence Score = 0.377
6: Perplexity = -8.213. Coherence Score = 0.407
7: Perplexity = -8.303. Coherence Score = 0.44
8: Perplexity = -8.513. Coherence Score = 0.4
9: Perplexity = -8.716. Coherence Score = 0.444
10: Perplexity = -9.048. Coherence Score = 0.438
11: Perplexity = -9.445. Coherence Score = 0.467
12: Perplexity = -9.942. Coherence Score = 0.538
13: Perplexity = -10.507. Coherence Score = 0.529
14: Perplexity = -11.019. Coherence Score = 0.511


Perplexity of 14 themes: -11.019
Coherence Score of 1 themes: 0.253

CPU times: user 2min 28s, sys: 716 ms, total: 2min 29s
Wall time: 2min 31s


In [ ]:
%time find_model(15, 25)

15: Perplexity = -11.365. Coherence Score = 0.51
16: Perplexity = -11.619. Coherence Score = 0.505
17: Perplexity = -11.868. Coherence Score = 0.483
18: Perplexity = -12.111. Coherence Score = 0.541
19: Perplexity = -12.366. Coherence Score = 0.476
20: Perplexity = -12.614. Coherence Score = 0.452
21: Perplexity = -12.86. Coherence Score = 0.482
22: Perplexity = -13.088. Coherence Score = 0.457
23: Perplexity = -13.326. Coherence Score = 0.438
24: Perplexity = -13.548. Coherence Score = 0.431


Perplexity of 24 themes: -13.548
Coherence Score of 24 themes: 0.431

CPU times: user 2min 51s, sys: 741 ms, total: 2min 52s
Wall time: 2min 53s


Оптимальное количество тем для метрики Perplexity score оказалось 14, а для метрики Coherence Score 1

Однако при увличении диапазона поиска, то есть от 15 до 25, оптимальным количеством оказалось 24 в обоих случаях

In [ ]:
# посмотрим, какие слова соответствуют каждой теме в лучшей модели
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=14,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)

print('\nPerplexity: ', round(lda_model.log_perplexity(corpus), 3))

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', round(coherence_lda, 3), '\n')

pprint(lda_model.print_topics())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -11.019

Coherence Score:  0.511 

[(0,
  '0.036*"страхование" + 0.022*"захотеть" + 0.016*"самым" + 0.015*"меньше" + '
  '0.014*"предприятие" + 0.011*"пристав" + 0.011*"угрожать" + '
  '0.009*"указывать" + 0.009*"судебный" + 0.007*"sms"'),
 (1,
  '0.000*"объективность" + 0.000*"международныи" + 0.000*"побег" + '
  '0.000*"перечеркнулось" + 0.000*"переводчик" + 0.000*"огорчить" + '
  '0.000*"получательница" + 0.000*"мучаетсяпалка" + 0.000*"кириллицеи" + '
  '0.000*"комсы"'),
 (2,
  '0.030*"валютный" + 0.011*"приятныи" + 0.003*"пустой" + 0.003*"дно" + '
  '0.000*"шестой" + 0.000*"терпеливо" + 0.000*"собственный" + 0.000*"десяток" '
  '+ 0.000*"подбор" + 0.000*"ежечастно"'),
 (3,
  '0.025*"закон" + 0.015*"александр" + 0.014*"ответственность" + 0.014*"бки" + '
  '0.012*"семья" + 0.011*"нарушение" + 0.011*"автоматически" + '
  '0.011*"здоровье" + 0.010*"данную" + 0.010*"группа"'),
 (4,
  '0.020*"взнос" + 0.019*"добрый" + 0.012*"рассчитывать" + 0.008*"беседа" + '
  '0.007*"данн

### Как бы вы назвали каждую тему? Обоснуйте.


Прежде всего, выберем ту модель, где соблюдается золотая середина, а именно кол-во тем = 8

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=8,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)

print('\nPerplexity: ', round(lda_model.log_perplexity(corpus), 3))

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', round(coherence_lda, 3), '\n')

pprint(lda_model.print_topics())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -8.513

Coherence Score:  0.4 

[(0,
  '0.037*"кредит" + 0.027*"сумма" + 0.027*"долг" + 0.016*"платить" + '
  '0.014*"рублеи" + 0.013*"помочь" + 0.012*"руб" + 0.010*"платёж" + '
  '0.010*"штраф" + 0.008*"процент"'),
 (1,
  '0.000*"объективность" + 0.000*"международныи" + 0.000*"побег" + '
  '0.000*"перечеркнулось" + 0.000*"переводчик" + 0.000*"огорчить" + '
  '0.000*"получательница" + 0.000*"мучаетсяпалка" + 0.000*"кириллицеи" + '
  '0.000*"комсы"'),
 (2,
  '0.028*"встреча" + 0.023*"представитель" + 0.010*"наличный" + '
  '0.010*"подписать" + 0.008*"залог" + 0.007*"регистрация" + 0.006*"приехать" '
  '+ 0.006*"сидеть" + 0.006*"профессионал" + 0.006*"прям"'),
 (3,
  '0.063*"банк" + 0.016*"тинькофф" + 0.014*"мой" + 0.013*"сотрудник" + '
  '0.011*"счёт" + 0.011*"вопрос" + 0.010*"клиент" + 0.008*"это" + 0.007*"год" '
  '+ 0.007*"ваш"'),
 (4,
  '0.010*"предприятие" + 0.007*"забывать" + 0.006*"удерживать" + '
  '0.006*"списаться" + 0.006*"случиться" + 0.005*"говориться" + 0.005

Тут сразу можно определить название дял темы 6 - "право", так как большинство слов относятся к этой категории.

Теме 1, возможно, подходит просто "банк", так как там большинство слов связано с деятельностью банка: карта, кредит, деньги, счет

Тема 2 связана с авто, из-за таких слов, как "подвести", "заправка", "предупреждать"

Тема 3 - "конфиденциальность" из-за синонимов (тайна, секретный) и связанных слов (записка, исполнительб комитет)

Тема 4 - "покупки", так как всеми содержащимися там словами можно охарактеризовать покупки в магазине (покупка может быть ужасной, что-то может пропасть (то есть украли) в магазине, продавец мог оказаться адекватным и не предлагать пакет)

Тема 5 - "отзыв тинька", так как все относится к определенному банку и показывает чье-то отношение к нему (усилительное наречие "очень", слово "пользоваться", слова благодарности)

Темы 0 и 7 сложно определить, так как там есть слова с опечатками, как, например, "тысруб" или просто числа в виде строки

### Визуализируйте темы с помощью пакета pyLDAvis.


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.276534 -0.053354       1        1  52.305205
7     -0.216781 -0.155011       2        1  20.673656
5     -0.161274  0.268467       3        1  18.460238
0      0.100517 -0.079535       4        1   5.991246
2      0.175424  0.009743       5        1   2.320390
4      0.130641  0.003419       6        1   0.202453
6      0.124514  0.003161       7        1   0.035040
1      0.123494  0.003111       8        1   0.011773, topic_info=           Term         Freq        Total Category  logprob  loglift
123       карта  1704.000000  1704.000000  Default  30.0000  30.0000
174      кредит   461.000000   461.000000  Default  29.0000  29.0000
84        сумма   339.000000   339.000000  Default  28.0000  28.0000
58        очень   819.000000   819.000000  Default  27.0000  27.0000
2          банк  4417.000000  4417.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
25     дядиньки     0.000871     0.665884   Topic8  -9.6866   2.4078
26     заведует     0.000871     0.665886   Topic8  -9.6866   2.4078
27      звонить     0.000871   315.745081   Topic8  -9.6866  -3.7538
28       звонок     0.000871   269.638715   Topic8  -9.6866  -3.5959
29   измениться     0.000871    18.530157   Topic8  -9.6866  -0.9183

[465 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         1  0.625199            ps
0         4  0.312600            ps
1         1  0.989714         адрес
1374      2  0.992186     активация
2291      2  0.985674  активировать
...     ...       ...           ...
165       1  0.403163           это
165       2  0.276455           это
165       3  0.320062           это
205       1  0.999804      являться
2573      1  0.993037        январь

[442 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 6, 1, 3, 5, 7, 2])

### Связаны ли темы отзывовов и поле “Score” в данных? Обоснуйте.

Вряд ли есть рпямая связь, так как в 1 теме, например, содержатся часто употребляемые слова в отзывах, например, слова "банк" или "сотрудник". Эти слова могут присутствовать как в положительном отзыве, так и в отрицательном.

Как видно из приведенных ниже таблиц, как в положительных, так и в отрицательных отзывах есть приведенные в пример слова

In [ ]:
positive_reviews = data_bank_lemma[(data_bank_lemma['score'] == '4') | (data_bank_lemma['score'] == '5')]
negative_reviews = data_bank_lemma[(data_bank_lemma['score'] == '1') | (data_bank_lemma['score'] == '2')]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
positive_reviews.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
6204,/services/responses/bank/tcs/,Vicky-13,\n\t\tУ меня счет ИП в этом банке. несколько д...,тинькофф банк,http://banki.ru,05.02.2019 18:13,NaN,5,Проверяется,/profile/?UID=230777
6205,/services/responses/bank/tcs/,loly,\n\t\tПользуюсь банком достаточно давно. Как к...,тинькофф банк,http://banki.ru,05.02.2019 18:09,NaN,5,Проверяется,/profile/?UID=125397
6206,/services/responses/bank/tcs/,9032489257,\n\t\tЯвляюсь клиентом банка как ИП и физ. лиц...,тинькофф банк,http://banki.ru,22.01.2019 13:58,NaN,5,Не засчитана,/profile/?UID=1579990
6208,/services/responses/bank/tcs/,b*******@mail.ru,\n\t\tОбщалась дважды со специалистом банка Е...,тинькофф банк,http://banki.ru,12.02.2019 12:21,NaN,5,Не засчитана,/profile/?UID=1687279
6210,/services/responses/bank/tcs/,маманаташа,\n\t\tПользуюсь кредитной картой. Баланс на не...,тинькофф банк,http://banki.ru,12.02.2019 12:27,NaN,5,Не засчитана,/profile/?UID=1687260


In [ ]:
negative_reviews.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
6215,/services/responses/bank/tcs/,Galina_11,\n\t\tОбратилась в банк за кредитом под залог ...,тинькофф банк,http://banki.ru,08.02.2019 13:15,NaN,2,Проверяется,/profile/?UID=1654898
6230,/services/responses/bank/tcs/,koval7874,"\n\t\tОбслуживаюсь в АО ""Тинькофф Банк"", как И...",тинькофф банк,http://banki.ru,07.02.2019 19:24,NaN,1,Проверяется,/profile/?UID=1663867
6234,/services/responses/bank/tcs/,9*******@mail.ru,\n\t\tБанк произвел безакцептное списание с де...,тинькофф банк,http://banki.ru,11.02.2019 18:10,NaN,1,Проверяется,/profile/?UID=1684153
6244,/services/responses/bank/tcs/,Neplim,"\n\t\tДоброго дня, подавал заявку на автокреди...",тинькофф банк,http://banki.ru,17.01.2019 2:36,NaN,1,Проверяется,/profile/?UID=1031685
6254,/services/responses/bank/tcs/,n*******@mail.ru,\n\t\tМне пришел стандартный штраф из Голланди...,тинькофф банк,http://banki.ru,14.01.2019 20:45,NaN,1,Проверяется,/profile/?UID=1542747
